In [29]:
import findspark 
findspark.init()

import pyspark 
from pyspark import SparkContext
#SparkContext è contenuto nella classe SparkSession

#MAP -> è una funzione di trasformazione
    #argomento -> un RDD
    #risultato -> un RDD -> passato al REDUCE o altra funzione
#REDUCE -> azione di sintesi -> può essere sostituita da una funzione

candidati=['Rossi M. 25','Bianchi A. 30','Verdi C. 18',
            'Marroni D. 05','Gialli E. 10',]
#Consideriamo che il voto è in 30esimi 
# e che corrisponde agli ultimi 2 caratteri

#CREIAMO LO SparkContex
    #getOrCreate() -> impedisce Context MULTIPLI
sc=SparkContext.getOrCreate()

# sc=SparkContext('local','esempio')


#CREIAMO L'RDD
rdd_candidati=sc.parallelize(candidati)

print("VOTO MINIMO -> map(LAMBDA).min() SENZA REDUCE")
#Troviamo il minimo 
    # min() usato al posto di una reduce
voto_min=rdd_candidati.map(lambda voto: int (voto[-2:])).min()
#Fa una map di tutti gli ultimi 2 caratteri per e dopo calcola il min()
    #la lambda ritorna interi
print(voto_min)

print("Ordinamento in base alla Chiave -> map(LAMBDA) e sortBy")

#Scorre l'RDD eeparia ogni elemento dell'RRD con una virgola"," 
    # contiene una sorta di Lista di liste
token=rdd_candidati.map(lambda x: x.split())
# [['Rossi', 'M.', '25'], 
#  ['Bianchi', 'A.', '30'], 
#  ['Verdi', 'C.', '18'], 
#  ['Marroni', 'D.', '05'], 
#  ['Gialli', 'E.', '10']]

#Scorre token e Separa Nome[0] e Cognome[1] dal Voto[2] 
    # ad ogni iterazione ritorna una tupla chiave/valore
                                        #f'({CHI}{AVE},{VALORE})'
key_value=token.map(lambda parti: (f'{parti[0]} {parti[1]}',parti[2]))
#nuovo risultato in key_value -> (Nome Cognome)[0] Voto[1].
#[('Rossi M.', '25'),                    #CHIAVE  / VALORE
# ('Bianchi A.', '30'), 
# ('Verdi C.', '18'), 
# ('Marroni D.', '05'), 
# ('Gialli E.', '10')]

#ordina tutto in modo decrescente in base al valore -> voto[1] di key_value
sorted_key_value=key_value.sortBy(lambda x: x[1], ascending=False)
#mette i dati ordinati in sorted_key

#CONVERTIAMO L'RDD degli ordinati IN UNA LISTA ris
ris=sorted_key_value.collect()
print(ris)

sc.stop()


VOTO MINIMO -> map(LAMBDA).min() SENZA REDUCE
5
Ordinamento in base alla Chiave -> map(LAMBDA) e sortBy
[('Bianchi A.', '30'), ('Rossi M.', '25'), ('Verdi C.', '18'), ('Gialli E.', '10'), ('Marroni D.', '05')]


In [31]:
import findspark 
findspark.init()

import pyspark 
from pyspark import SparkContext

numeri=[2,9,16,57,95,3,1,51,16,856,1,8,633,9,7,1,45,6]

sc=SparkContext.getOrCreate()

# sc=SparkContext('local','esempio')


#CREIAMO L'RDD
rdd_numeri=sc.parallelize(numeri)

print("VOTO MINIMO -> map(LAMBDA).min() SENZA REDUCE")

#Crea coppie->tuple chiave/valore stringa/numero
# in cui associamo DISPARI se il numero è dispari altrimenti PARI se è pari
pari_dispari=rdd_numeri.map(lambda n:('DISPARI' if n%2 else 'PARI',n))
#[('PARI', 2), ('DISPARI', 9),  ('PARI', 16), 
# ('DISPARI', 57), ('DISPARI', 95), ('DISPARI', 3), 
# ('DISPARI', 1), ('DISPARI', 51), ('PARI', 16), 
# ('PARI', 856), ('DISPARI', 1), ('PARI', 8), 
# ('DISPARI', 633), ('DISPARI', 9), ('DISPARI', 7), 
# ('DISPARI', 1), ('DISPARI', 45), ('PARI', 6)]


#Raccoglie le coppie per chiave a cui il valore associa a e b
    # -> a <- somma parziale
    # -> b <- valore chie viene sommato via via ad a
reduced_pari_dispari=pari_dispari.reduceByKey(lambda a,b: a+b)
#[('DISPARI', 912), ('PARI', 904)]

ris=reduced_pari_dispari.collect()
print(ris)


sc.stop()


VOTO MINIMO -> map(LAMBDA).min() SENZA REDUCE
[('PARI', 2), ('DISPARI', 9), ('PARI', 16), ('DISPARI', 57), ('DISPARI', 95), ('DISPARI', 3), ('DISPARI', 1), ('DISPARI', 51), ('PARI', 16), ('PARI', 856), ('DISPARI', 1), ('PARI', 8), ('DISPARI', 633), ('DISPARI', 9), ('DISPARI', 7), ('DISPARI', 1), ('DISPARI', 45), ('PARI', 6)]
[('DISPARI', 912), ('PARI', 904)]
[('DISPARI', 912), ('PARI', 904)]
